In [16]:
import pandas as pd
df = pd.read_csv('public.csv')

## Ajuste de dados e preenchimento de Imagens e Metar nulos

In [17]:
import re
# Preenchendo dados reais nulos por dados previstos
    # Alguns casos podem nao possuir tanto dados reais quanto previstos
df.loc[df['metar'].isna(), 'metar'] = df.loc[df['metar'].isna(), 'metaf']

# Substituir o texto METAF por METAR
df['metar'] = df['metar'].apply(lambda txt: re.sub('METAF', 'METAR', str(txt)))

# Funcao para ajustar dados meteorologicos com falta de informacao temporal
def metar_fix(txt):
    txt = str(txt)
    if txt != None and txt != 'nan':  
        first_position = txt.find('SB')
        second_position = txt[first_position+5:].find(' ')
        final_position = first_position+5+second_position
        if txt[final_position-1] != 'Z':
            txt = txt[:final_position] + 'Z' + txt[final_position:]
        return txt

df['metar'] = df['metar'].apply(metar_fix)

In [18]:
# Funcao para preencher as urls de imagem nulas baseado na proximidade por data, threshold de 3 horas 
from datetime import timedelta

df['hora_ref'] = pd.to_datetime(df['hora_ref'])

def find_nearest_image(date, df_full):
    window = timedelta(hours=3)
    datas_proximas = (df_full['hora_ref']-date).abs()
    if (datas_proximas <= window).sum() > 0:
        nearest_row = df_full.iloc[datas_proximas.argsort()[0]]
        return nearest_row['url_img_satelite']
    
df_full = df.loc[~df['url_img_satelite'].isna()].drop_duplicates()

df.loc[df['url_img_satelite'].isna(), 'url_img_satelite'] = df.loc[df['url_img_satelite'].isna(), 'hora_ref'].apply(lambda x: find_nearest_image(x, df_full))

In [19]:
# Funcao para preencher o metar baseado na proximidade por data e aeroporto, threshold de 3 horas 

def find_nearest_metar(date, airport, df_full):
    # print(date, airport)
    window = timedelta(hours=3)
    datas_proximas = (df_full.loc[df_full['origem'] == airport, 'hora_ref']-date).abs()
    if (datas_proximas <= window).sum() > 0:
        nearest_row = df_full.iloc[datas_proximas.argsort().tolist()[0]]
        return nearest_row['metar']
    return None

df_full = df.loc[~df['metar'].isna()].drop_duplicates()

df.loc[df['metar'].isna(), 'metar'] = df.loc[df['metar'].isna(), ['hora_ref', 'origem']].apply(lambda x: find_nearest_metar(x[0], x[1], df_full), axis=1)

C:\Users\pedro\AppData\Local\Temp\ipykernel_15736\2912340457.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.loc[df['metar'].isna(), 'metar'] = df.loc[df['metar'].isna(), ['hora_ref', 'origem']].apply(lambda x: find_nearest_metar(x[0], x[1], df_full), axis=1)
C:\Users\pedro\AppData\Local\Temp\ipykernel_15736\2912340457.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.loc[df['metar'].isna(), 'metar'] = df.loc[df['metar'].isna(), ['hora_ref', 'origem']].apply(lambda x: find_nearest_metar(x[0], x[1], df_full), axis=1)
C:\Users\pedro\AppData\Local\Temp\ipykernel_15736\2912340457.py:14: FutureWarning: Serie

## Criacao do conjuto de treino

In [20]:
# Importando a tabela de metar construida em metar.ipynb
df_metar = pd.read_csv('metar.csv')

# Removendo os casos de metar e imagens nulos restantes (nao fazem parte do conjunto de teste)
df_model = df.loc[~df['metar'].isna() & ~df['url_img_satelite'].isna()]

# Isolando o nome do arquivo de imagem
df_model['img'] = df_model['url_img_satelite'].apply(lambda txt: txt.split('/')[-1][:-4])

C:\Users\pedro\AppData\Local\Temp\ipykernel_15736\1112468669.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['img'] = df_model['url_img_satelite'].apply(lambda txt: txt.split('/')[-1][:-4])


In [21]:
# Criando uma tabela de latitude e longitude para os aeroportos
df_latitudes = df_metar[['station_id', 'latitude', 'longitude']].drop_duplicates()

# Jutando o dataframe orignal com o de metar, removendo dados que nao serao utilizados
df_model = df_model.merge(df_metar, how='left', left_on='metar', right_on='meta')
df_model.drop(['hora_ref', 'station_id', 'meta', 'metar', 'metaf', 'date', 'latitude', 'longitude'], axis=1, inplace=True)

# Inserindo as latidudes e longitudes para a origem e destino, e removendo os mesmos
df_model = df_model.merge(df_latitudes, how='left', left_on='origem', right_on='station_id').rename(columns={'latitude':'origem_latitude', 'longitude':'origem_longitude'}).drop(['station_id'], axis=1)
df_model = df_model.merge(df_latitudes, how='left', left_on='destino', right_on='station_id').rename(columns={'latitude':'destino_latitude', 'longitude':'destino_longitude'}).drop(['station_id'], axis=1)
# df_model.drop(['origem', 'destino'], axis=1, inplace=True)

In [22]:
# Separacao do conjunto de treino e de teste
df_test = df_model.loc[df_model['espera'].isna()]
df_train = df_model.loc[~df_model['espera'].isna()]

In [23]:
df_model.to_csv('tabelacompleta.csv', index=False)